# ДЗ Линейная регрессия

В данном задании мы рассмотрим набор данных об учащихся, собранный в 2006 году в одной из школ Португалии. Данные представлены в неудобном для машинного обучения виде, и содержат мусор. Ваша задача &mdash; привести их к надлежащему виду и обучить на них простую модель.

Данные состоят из четырех файлов:
- data.csv &mdash; основная таблица с информацией о учащихся
- scores.csv &mdash; список финальных оценок по одному из предметов (20-балльная шкала переведенная в проценты)
- attendance.csv &mdash; таблица посещений занятий по этому предмету
- school_support.txt &mdash; список учащихся, которым оказывается финансовая поддержка

Ваша задача &mdash; построить модель для предсказания финальных оценок исходя из всех остальных данных и проверить качество ее работы с помощью кросс-валидации. В качестве алгоритма мы будем использовать линейную регрессию

Расшифровка столбцов в data.csv для справки:
- age &mdash; возраст
- Medu &mdash; уровень образования матери (по некоторой условной шкале)
- Fedu &mdash; уровень образования отца (по некоторой условной шкале)
- traveltime &mdash; время в пути до школы (1 – < 15 мин., 2 – от 15 до 30 мин., 3 – от 30 мин. to 1 ч.
или 4 – > 1 ч.)
- studytime &mdash; время, затрачиваемое на занятия вне школы (1 – < 2 ч., 2 – от 2 до 5 ч., 3 – от 5 до 10 ч. или 4 – > 10 ч.)
- famrel &mdash; насколько хорошие отношения в семье у учащегося (по некоторой условной шкале)
- freetime &mdash; количество свободного времени вне школы (по некоторой условной шкале)
- goout &mdash; время, затрачиваемое на общение с друзьями (по некоторой условной шкале)
- Dalc &mdash; количество употребления алкоголя в учебные дни (по некоторой условной шкале)
- Walc &mdash; количество употребления алкоголя в неучебные дни (по некоторой условной шкале)
- health &mdash; уровень здоровья (по некоторой условной шкале)
- sex_M &mdash; пол: мужской (1) или женский (0)
- address_U &mdash; живет ли учащийся в городе (1) или в пригороде (0)
- famsize_LE3 &mdash; размер семьи: не больше 3 человек (1) или больше (0)
- Pstatus_T &mdash; живут ли родители вместе (1) или отдельно (0)
- nursery &mdash; посещал ли учащийся детский сад
- plans_university &mdash; планирует ли учащийся поступать в университет (-1 или 1)
- past_failures &mdash; количество неудовлетворительных оценок по другим предметам ранее (от 0 до 4)

*Примечание. Несколько признаков в данных содержат ошибки/проблемы/некорректности. Эти проблемы нужно исправить. Для
проверки &mdash; всего в данных таких проблем четыре.*

### Задача 1: сломанный признак (а может и не один)
__(1 балл)__

Загрузите таблицу data.csv.

Найдите в данных сломанный признак (он не соответствует описанию) и исправьте его.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

data = pd.read_csv('data.csv')

check = []

rows_of_all_table = 649

s = 'plans_universitypast_failures'

for i in range (rows_of_all_table):
  if(data.loc[i, s] < 0):
    check.append(True)
    data.loc[i, s] *= -1
    data.loc[i, s] = str(data.loc[i, s])
    data.loc[i, s] = data.loc[i, s][0] + ' ' + data.loc[i, s][1]
  else :
    check.append(False)
    data.loc[i, s] = str(data.loc[i, s])
    data.loc[i, s] = data.loc[i, s][0] + ' ' + data.loc[i, s][1]

#print(data)


new_df = data[s].str.split(' ',expand=True)

new_df.columns=['plans_university','past_failures']

data = pd.concat([data,new_df],axis=1)

data = data.drop(s,axis=1)

a = 'plans_university'
b = 'past_failures'

for i in range (rows_of_all_table):
  data.loc[i, b] = int(data.loc[i, b])
  data.loc[i, a] = int(data.loc[i, a])
  if check[i] == True:
    data.loc[i, a] *= -1

In [9]:
for i in range (rows_of_all_table):
  if(data.loc[i, 'age'] > 1750):
    data.loc[i, 'age'] = 2006 - data.loc[i, 'age']

### Задача 2: пропуски в данных
__(1 балл)__

Проверьте, есть ли в данных пропуски (значения NaN). Замените все пропущенные значения на среднее значение этого признака по столбцу.

__(+1 балл)__

Дополнительно сравните качество замены на среднее по столбцу и на медиану по столбцу


*Hint: изучите в pandas функции loc, isnull, а также передачу булевых массивов в качестве индексов.*

In [5]:
data_columns_before = data.columns

array_of_nuns = pd.DataFrame()

for i in data.columns:
  sum = 0
  c = 0
  bool_series = pd.isnull(data[i])
  array_of_nuns[i] = bool_series
  for j in range(rows_of_all_table):
    if bool_series[j] == False:
      sum += data.loc[j, i]
      c += 1
  avg = round(sum / c)
  for j in range(rows_of_all_table):
    if bool_series[j] == True:
      data.loc[j, i] = avg

In [15]:
# take median to NaN

for i in data_columns_before:
  bool_series = data[i].values is not None
  a = []
  for j in range(rows_of_all_table):
    if array_of_nuns.loc[j, i] == False:
      a.append(data.loc[j, i])
  a.sort()
  n = len(a)
  for j in range(rows_of_all_table):
    if array_of_nuns.loc[j, i] == False:
      data.loc[j, i] = a[n // 2]

print(data)

          age  Medu  Fedu  traveltime  studytime  famrel  freetime  goout  \
0    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
1    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
2    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
3    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
4    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
..        ...   ...   ...         ...        ...     ...       ...    ...   
644  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
645  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
646  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
647  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
648  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   

     Dalc  Walc  health  sex_M  address_U  famsize_LE3  Pstatus_T  nursery 

### Задача 3: нормализация данных
__(1 балл)__

Нормализуйте данные любым способом

In [13]:
for i in data.columns:
  mn = 1e9
  mx = -1e9
  for j in range(rows_of_all_table):
    mx = max(mx, data.loc[j, i]);
    mn = min(mn, data.loc[j, i]);
  for j in range(rows_of_all_table):
    data.loc[j, i] = (data.loc[j, i] - mn) / (mx - mn)

### Задача 4: кросс-валидация для исходных данных
__(1 балл)__

Загрузите файл scores.csv и протестируйте, как линейная регрессия предсказывает ответ сейчас (с помощью кросс-валидации).

*Hint: воспользуйтесь sklearn.linear_model и sklearn.model_selection.*

In [6]:
scores = pd.read_csv('scores.csv')
X = data.values
y = scores.values
model = LinearRegression()
res = cross_validate(model, data.values, scores.values, cv = 4)
res['test_score']





array([0.21881645, 0.25841768, 0.14161857, 0.23287806])

### Задача 5: полные данные
__(2 балла)__

Воспользуйтесь файлами attendance.csv и school_support.txt для того, чтобы добавить новые признаки в данные. Желательно по максимуму использовать возможности pandas для упрощения преобразований.

school_suport число в строке значит что i-ый школьник из исходной таблицы получал мат помощь (обратите внимание что строк в файле меньше, подумайте как правильно импортировать данные)

Попробуйте несколько способов добавления полных данных

In [7]:
attend = pd.read_csv('attendance.csv', delimiter = ";")
support = pd.read_table('school_support.txt')

a = []

for j in range(rows_of_all_table):
  cnt = 0
  for i in attend.columns:
    if attend.loc[j, i] == '+':
      cnt += 1
  a.append(cnt)

data['attend'] = a

a = [0] * rows_of_all_table

for i in range(len(support)):
  a[support.loc[i, 'support']] = 1

data['support'] = a

### Задача 6: борьба с выбросами
__(1.5 балла)__

Качество предсказания может ухудшаться, если в данных присутствуют корректные значения признаков (с точки зрения чтения данных и применения методов), но не соответствующие реальным объектам. Например, данные могли быть введены в неверном формате, а потом слишком грубо приведены к общему виду, из-за чего ошибка не была замечена.
Попробуем от такого избавиться &mdash; а для этого такие объекты нужно сначала найти. Конечно, нам еще недоступны многие продвинутые способы, но давайте попробуем обойтись простыми.

Первый способ это сделать &mdash; посмотреть для каждого признака на распределение его значений и проверить крайние значения на правдоподобность. (постройте гистограммы для признаков, как минимум для подозрительных)

*Hint 1: используйте функцию DataFrame.hist*

*Hint 2: в описании датасета выше есть информация, необходимая для восстановления правильных значений*

In [8]:
# for i in data.columns:
#   data.hist(column=f'{i}')

s = 'traveltime'

for i in range(rows_of_all_table):
  if(data.loc[i, s] > 4):
    if(data.loc[i, s] < 15):
      data.loc[i, s] = 1
    elif data.loc[i, s] < 30:
      data.loc[i, s] = 2
    elif data.loc[i, s] < 60:
      data.loc[i, s] = 3
    else :
      data.loc[i, s] = 4


__(1.5 балла)__

Другой простой способ найти выбросы &mdash; сделать предсказаниепосчитать ошибку на каждом объекте по отдельности и посмотреть на объекты с наибольшей ошибкой. Обратите внимание, что просто удалять все объекты с высокой ошибкой нельзя &mdash; это, конечно, хороший способ добиться меньшей ошибки (на данной выборке), но одновременно вы ухудшите обобщающую способность алгоритма. Вместо этого вам нужно найти однозначно ошибочные записи и их исправить.

*Hint: возможно, все проблемы уже были найдены первым способом; для проверки &mdash; в сумме здесь нужно исправить 3 проблемы.*

Для поиска ошибки на одном отдельном обьекте придётся обучить линейную регрессию руками. Частичный пример, допишите код. Постройте гистограмму распределения ошибок

In [12]:
import sklearn
from sklearn import linear_model
X1 = data.values
y1 = scores.values
regression = linear_model.LinearRegression().fit(X1, y1)
error = []
for i in X1:
  j = [i]
  prediction = regression.predict(j)
  error.append((prediction[0] - y)**2)
array_of_errors = []
for i in error[0]:
  array_of_errors.append(i[0])

array_of_errors.sort()
array_of_errors.reverse()
print(array_of_errors)

[4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4102.0523520078905, 4057.341765425726, 4038.2550854619412, 4031.902858807347, 4031.902858807347, 4012.8761788435613, 3486.5796865483967, 1524.6890247104202, 1159.216359250926, 1159.216359250926, 1159.216359250926, 958.0717082775025, 958.0717082775025, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 843.743693791432, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 673.5443737369966, 578.2710283319378, 5

Все ошибки были исправлены заранее, ошибки в предсказании оценок являются нормальными

### Финальное предсказание и отчёт

Проведите предсказание еще раз и сравните качество с исходным. Запишите свои наблюдения - как изменялось качество обучения модели при использовании разных модификаций данных.

###Запуск со средним значением из 2 задачи

In [14]:
scores = pd.read_csv('scores.csv')
X = data.values
y = scores.values
model = LinearRegression()
res = cross_validate(model, data.values, scores.values, cv = 4)
res['test_score']

array([0.27060316, 0.2757387 , 0.12815065, 0.22059342])

Запуск с медианным значением из второй задачи

In [16]:
print(data)
scores = pd.read_csv('scores.csv')
X = data.values
y = scores.values
model = LinearRegression()
res = cross_validate(model, data.values, scores.values, cv = 4)
res['test_score']

          age  Medu  Fedu  traveltime  studytime  famrel  freetime  goout  \
0    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
1    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
2    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
3    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
4    0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
..        ...   ...   ...         ...        ...     ...       ...    ...   
644  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
645  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
646  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
647  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   
648  0.285714   0.5   0.5         0.0   0.333333    0.75       0.5    0.5   

     Dalc  Walc  health  sex_M  address_U  famsize_LE3  Pstatus_T  nursery 

array([-0.05207104,  0.00912757, -0.01473885, -0.00685901])

###Отчет
Качество обучения при модифицированных данных поменялось не сильно(видимо из-за небольшого объема неверных данных по сравнению со всеми данными), задание было прикольное, мне понравилось, делал вместе с Артемом Багринцевым. К сожалению не получилось нормально запустить с медианным значением(не смогли найти багу)